In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
pd.set_option('display.max_columns', 5000)

In [6]:
#compares a player with the mean of his position
def comparison(playerid, matchId, file, cols):
    total=pd.read_csv(file)
    m=total[cols].mean()
    print('The mean:')
    print(m)
    total=total[total['playerId']==playerid]
    total=total[total['matchId']==matchId]
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    print('player:', dfw['player.name'].iloc[0])
    print(total[cols])
    

In [7]:
#returns the player using player and match id
def get_player(playerid, matchId):
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    return dfw['player.name'].iloc[0]

In [8]:
#print the top 50 players according to the ranking
def bestplayers(df):
    for i in range (50):
        id=df['playerId'].iloc[i]
        matchid=df['matchId'].iloc[i]
        player=get_player(id,matchid)
        print('matchid:',matchid)
        print('playerid:', id)
        print(player)

In [9]:
#negative features are multiplied by -1 for the convention
loss_list= [
    'total.losses',
    'total.ownHalfLosses',
    'total.dangerousOwnHalfLosses'
]

In [50]:
#preprocess and filter the dataframe with the feature set 
def preprocessdf(mean, std, position):
    meandf=pd.read_csv(mean)
    stddf=pd.read_csv(std)
    pdf=pd.read_csv(position)
    meandf=meandf[meandf.columns[1:]]
    stddf=stddf[stddf.columns[1:]]
    '''
    pdf=pdf[pdf.columns[1:]]
    '''
    keep_columns=meandf.columns
    calc_col = [col for col in pdf.columns if col.startswith('total.') and col != 'total.minutesOnField']
    calc_filtered = [col for col in calc_col if col not in keep_columns]
    cpdf=pdf.drop(columns=calc_filtered)
    for w in loss_list:
        if(w in keep_columns):
            cpdf[w]*=-1
    return cpdf, meandf, stddf

In [11]:
#calculate the weighted sum of the data normalized to the selected cluster from the result
def resultcalculation(calc,mean,std,i, weight):
    t0=calc.copy()
    
    t0[mean.columns]-=mean.loc[i]
    
    t0[mean.columns]/=std.loc[i]

    vec=np.array(weight)
    calc_col = [col for col in calc.columns if col.startswith('total.') and col != 'total.minutesOnField']
    statsarray0=t0[calc_col].to_numpy()
    res0=np.dot(statsarray0, vec)
    return res0

In [118]:
meanpath='mean_std/mean_stv3clustering.csv'
stdpath='mean_std/std_stv3clustering.csv'
playerpath='norm_stats/normalized_striker.csv'

In [119]:
pd.read_csv(meanpath)

,Unnamed: 0,total.goals,total.shots,total.keyPasses,total.shotAssists,total.recoveries,total.opponentHalfRecoveries,total.losses,total.ownHalfLosses,total.xgShot,total.xgAssist,total.receivedPass,total.touchInBox,total.progressivePasses,total.shotsOnTarget
0,0,0.406524,0.637292,-0.272048,-0.232560,-0.553720,-0.465354,0.033788,0.157258,0.480186,-0.281279,0.016407,0.488706,-0.329428,0.612906
1,1,-0.320824,-0.442419,-0.131503,-0.073640,0.685936,0.496176,-0.599109,-0.622552,-0.406975,-0.159202,-0.044779,-0.395813,0.337852,-0.424786
2,2,-0.441474,-0.508442,-0.222245,-0.274371,-0.408421,-0.332208,0.795881,0.625220,-0.435311,-0.297816,-0.425898,-0.421798,-0.406984,-0.483749
3,3,0.017723,-0.071204,0.016445,-0.024496,-0.158881,-0.140521,-0.225830,-0.233655,-0.080834,-0.052462,-0.083287,-0.087067,-0.084544,-0.062303


In [120]:
calcdf, meandf, stddf=preprocessdf(meanpath, stdpath, playerpath)

In [ ]:
calcdf

In [124]:
#weight vector for each player profile
iv_def_weights=[1,2,1,4,1,4,1,4,2,1,4,1,2,4,1,2,4,1,0,1,1]
iv_hyb_weights=[1,2,1,4,1,4,1,4,2,1,4,1,4,1,1,2,4,1,0,1,2]
iv_off_weights=[1,4,1,4,1,1,1,4,4,1,4,1,4,1,2,2,4,2,2,1,2]
wi_pla_weights=[1,1,4,1,1,4,2,1,4,2,4,4,2,4,4]
wi_dri_weights=[1,1,4,1,4,2,4,1,4,4,4,2,4,4,2]
wi_run_weights=[1,1,4,1,2,2,4,1,4,4,4,1,4,4,4]
st_run_weights=[4,1,0,2,2,4,2,1,4,4,4,4,1,4]
st_goa_weights=[4,1,0,2,2,4,2,4,4,4,2,2,1,4]
st_box_weights=[4,2,0,2,2,2,1,1,4,2,2,4,1,4]
st_fa9_weights=[4,2,0,2,2,4,2,4,4,4,2,2,2,4]
mi_dpl_weights=[1,4,0,1,4,4,1,4,1,2,1,1,2,4,4,1,1,2,4,0,2,4,0,1]
mi_opl_weights=[0,0,4,0,0,4,1,4,0,0,4,0,0,0,2,1,1,0,0,4,4,2,2,0]
mi_def_weights=[1,4,0,1,2,2,1,4,1,2,1,1,1,4,4,2,4,2,4,0,2,4,0,2]
mi_off_weights=[0,0,4,1,4,1,1,4,0,0,1,0,0,2,2,2,4,0,0,2,4,2,2,4]
lb_def_weights=[1,2,1,4,1,4,1,4,1,1,1,2,4,1,4,1,2,4,1,1,4,1,0,0,1,1]
lb_off_weights=[1,4,1,2,1,2,2,4,2,4,1,4,2,1,2,1,2,2,4,2,2,1,4,4,1,1]
lb_run_weights=[1,4,1,4,1,4,1,2,1,4,1,4,2,1,4,1,2,2,4,1,2,1,4,4,1,2]
unitlist=[1]*len(mi_def_weights)

In [132]:
#calculate the ratings based on the weights of the profile
r0=resultcalculation(calcdf,meandf,stddf, 3 , st_fa9_weights)
r1=resultcalculation(calcdf,meandf,stddf, 0 , st_box_weights)
r2=resultcalculation(calcdf,meandf,stddf, 1 , st_run_weights)
r3=resultcalculation(calcdf,meandf,stddf, 2 , st_goa_weights)

In [133]:
calcdf['false 9']=r0
calcdf['box']=r1
calcdf['2nd runner']=r2
calcdf['target']=r3

In [136]:
calcdf.to_csv('final_st_rating.csv' , index=False)